In [ ]:
%reset -f

In [ ]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

# Import

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import miceforest as mf
import random
import datetime

from deps.utils import *

from deps.prevent_equations import *

from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from deps.utils import *
import shap
from deps.personalized_survival import *
from deps.cox_regression_modelling import *
from deps.extract_KM_curve import *

from sklearn.metrics import make_scorer

from typing import List, Dict, Callable, Union, Tuple
import networkx as nx

import plotly
from pandas import DataFrame, Series
from scipy.stats import mannwhitneyu

from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot
from networkx import Graph, degree, get_edge_attributes
from networkx.classes.reportviews import DegreeView
from pandas import DataFrame
from toolz import identity

from IPython.display import display, HTML


import networkx
from community import community_louvain
from random import randint

from math import pi

import matplotlib.ticker as ticker
from typing import List, Dict, TypedDict, Tuple, Optional, Any

# Data

In [ ]:
path_folder = "" # change this to your path
out_path_folder = "" # change this to your path

results_path = "" # change

save_results_km = f"" # change

df_baseline = pd.read_excel("") # change

# Declarations and transformations

In [ ]:
SEED = 1000
random.seed(SEED)

sex_col_name = "sex_female0"
age_col_name = "age"
bmi_col_name = 'bmi'
male_code = 1 
female_code = 0

outcomes_defs = ["acv2", "tacv2_y"]
bin_event_var = outcomes_defs[0]
time_event_var = outcomes_defs[1]

event_out_var = outcomes_defs[0]
time_out_var = outcomes_defs[1]

proteins_cols = [] # change to list with proteins names

In [ ]:
eprime_avg = "" # change this to their respective column name 
e_a_ratio = ""
e_eprime_ratio = ""
la_strain = ""
lavi = ""
lvmi = "" 
rwt = ""

# A. Dimensionality reduction with supervised machine learning

In [ ]:
plot_shap = True
num_mice_datasets = 5
outcomes_defs = [] # list with 1) events column and 2) time to event column

## GBSA

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    "model__n_estimators": [50, 100,],   # Number of boosting stages
    "model__learning_rate": [0.05, 0.1, 0.2],  # Learning rate
    "model__max_depth": [3, 6],  # Tree depth
    "model__min_samples_leaf": [2, 5, 10]  # Minimum samples in each leaf
}

pipeline = Pipeline([
    ("scaler", StandardScaler()), 
    ("model", GradientBoostingSurvivalAnalysis(random_state=SEED))
])

In [ ]:
bin_event_var = outcomes_defs[0]
time_event_var = outcomes_defs[1]

df = df_baseline[proteins_cols+[bin_event_var, time_event_var]]

y = np.array([(event, time) for event, time in zip(df[bin_event_var], df[time_event_var])], dtype=[("event", bool), ("time", float)])

X = df[proteins_cols]

X = impute_input(X, num_mice_datasets, SEED)

### Getting best hyperparameters
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y['event'], random_state=SEED)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=c_index_scorer,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

compute_c_metrics(grid_search, X_train, X_test, y_train, y_test)

### Rebuild model with all data
# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

feature_names = proteins_cols
df_X_scaled = pd.DataFrame(X_scaled, columns=proteins_cols)

model = GradientBoostingSurvivalAnalysis(learning_rate = grid_search.best_params_["model__learning_rate"],
                                        max_depth = grid_search.best_params_["model__max_depth"],
                                        n_estimators = grid_search.best_params_["model__n_estimators"],
                                        min_samples_leaf = grid_search.best_params_["model__min_samples_leaf"],
                                        random_state=SEED
                                        ).fit(df_X_scaled, y)

# Evaluate best model using Concordance Index
c_index = model.score(df_X_scaled, y)
print(f"Optimized Concordance Index (all): {c_index:.3f}")

### Plot SHAP
if plot_shap:
    # Reobtain model to avoid warnings
    model_rawX = GradientBoostingSurvivalAnalysis(learning_rate = grid_search.best_params_["model__learning_rate"],
                                        max_depth = grid_search.best_params_["model__max_depth"],
                                        n_estimators = grid_search.best_params_["model__n_estimators"],
                                        min_samples_leaf = grid_search.best_params_["model__min_samples_leaf"],
                                        random_state=SEED
                                        ).fit(X_scaled, y)
    explainer = shap.Explainer(model_rawX.predict, X_scaled, feature_names=feature_names, seed=SEED)
    shaps = explainer(X_scaled)
    shap.plots.beeswarm(shaps, max_display=24)

In [ ]:
# Calculate mean absolute SHAP values for feature ranking
feature_importance = np.abs(shaps.values).mean(axis=0)

# Create a DataFrame for ranking
feature_ranking = pd.DataFrame({'Feature': X.columns, 'SHAP Importance': feature_importance})
feature_ranking = feature_ranking.sort_values(by='SHAP Importance', ascending=False)

upper_75 = feature_ranking["SHAP Importance"].describe().loc['75%']
top_feat_shap = feature_ranking[feature_ranking["SHAP Importance"]>upper_75]
display(top_feat_shap)
top_gbsa = list(top_feat_shap['Feature'])

## RSF

In [ ]:
random.seed(SEED)

# Define parameter grid for hyperparameter tuning
param_grid = {
    "model__n_estimators": [50, 100],   # Number of boosting stages
    "model__max_depth": [3, 6],  # Tree depth
    "model__min_samples_leaf": [2, 5, 10]  # Minimum samples to split a node
}

pipeline = Pipeline([
    ("scaler", StandardScaler()),  # Feature scaling inside each fold
    ("model", RandomSurvivalForest(random_state=SEED))
])

In [ ]:
df = df_baseline[proteins_cols+[bin_event_var, time_event_var]]

y = np.array([(event, time) for event, time in zip(df[bin_event_var], df[time_event_var])], dtype=[("event", bool), ("time", float)])

X = df[proteins_cols]

X = impute_input(X, num_mice_datasets, SEED)


### Getting best hyperparameters
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y['event'], random_state=SEED)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=c_index_scorer,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

compute_c_metrics(grid_search, X_train, X_test, y_train, y_test)

### Rebuild model with all data
# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

feature_names = proteins_cols
df_X_scaled = pd.DataFrame(X_scaled, columns=proteins_cols)

model = RandomSurvivalForest(
                                        max_depth = grid_search.best_params_["model__max_depth"],
                                        n_estimators = grid_search.best_params_["model__n_estimators"],
                                        min_samples_leaf = grid_search.best_params_["model__min_samples_leaf"],
                                        random_state=SEED
                                        ).fit(df_X_scaled, y)

# Evaluate best model using Concordance Index
c_index = model.score(df_X_scaled, y)
print(f"Optimized Concordance Index (all): {c_index:.3f}")

### Plot SHAP
if plot_shap:
    # Reobtain model to avoid warnings
    model_rawX = RandomSurvivalForest(
                                        max_depth = grid_search.best_params_["model__max_depth"],
                                        n_estimators = grid_search.best_params_["model__n_estimators"],  
                                        min_samples_leaf = grid_search.best_params_["model__min_samples_leaf"],
                                        random_state=SEED
                                        ).fit(X_scaled, y)
    explainer = shap.Explainer(model_rawX.predict, X_scaled, feature_names=feature_names, seed=SEED)
    shaps = explainer(X_scaled)
    shap.plots.beeswarm(shaps, max_display=24)

In [ ]:
# Calculate mean absolute SHAP values for feature ranking
feature_importance = np.abs(shaps.values).mean(axis=0)

# Create a DataFrame for ranking
feature_ranking = pd.DataFrame({'Feature': X.columns, 'SHAP Importance': feature_importance})
feature_ranking = feature_ranking.sort_values(by='SHAP Importance', ascending=False)

upper_75 = feature_ranking["SHAP Importance"].describe().loc['75%']
top_feat_shap = feature_ranking[feature_ranking["SHAP Importance"]>upper_75]
display(top_feat_shap)
top_rsf = list(top_feat_shap['Feature'])

## Output from outcomes analysis

In [ ]:
intersect_features = list(set(top_gbsa).intersection(set(top_rsf)))
intersect_features.sort()
selected_proteins = intersect_features.copy()

## Network

In [ ]:
cols_interest = proteins_cols.copy()

cols_interest_b = cols_interest 
node_color_set = ['black'] * len(cols_interest)

data_biomarkers_set = df_baseline[cols_interest]
data_biomarkers_set.columns = cols_interest_b

definedNodeColors = False

data_biomarkers_set.dropna(how='any')
print(len(data_biomarkers_set))

POWERS = range(2,3)

greaterThan = False
for power in POWERS:
    nodes = list(data_biomarkers_set.columns)
    adjacency_matrix = data_biomarkers_set.corr(method='spearman')
    print(nodes)
    graph = make_graph_from_adjacency_matrix(pd.DataFrame(adjacency_matrix, columns=nodes, index=nodes).pow(power))
    graph_position = nx.spring_layout(graph, seed=10, k=0.04)
    if greaterThan:
        annot_labels = adjacency_matrix[adjacency_matrix.abs() >= 0.03]
        annot_labels.fillna(0, inplace=True)
        annot_labels = annot_labels.to_numpy()
        annot_labels
        graph = make_graph_from_adjacency_matrix(DataFrame(annot_labels, columns=nodes, index=nodes).pow(power))
        graph_position = nx.spring_layout(graph, seed=5, k=10.45)
    edges, weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    degree_centrality = dict(nx.degree(graph, weight='weight'))
    nodes = degree_centrality.keys()
    graph_figsize_arg = dict(figsize=(25, 40), layout='constrained')
    pyplot.figure(**graph_figsize_arg)
    draw_graph(
        graph, graph_position, edge_width_scale=20, node_width_scale=100,
        box_background='#00000000', 
        modules_colors= sns.color_palette("colorblind"),
        font_color={frozenset(data_biomarkers_set.columns): '#3d3d3d'},
        highlight_markers=selected_proteins,
        render = ('graph')
    )
    pyplot.show()

# B. Phenogrouping derivation by unsupervised machine learning

## GMM

In [ ]:
X = df_baseline[proteins_cols]

X_clust = X[selected_proteins]

scaler = StandardScaler()
X_clust_scaled = scaler.fit_transform(X_clust)

covariance_type = "diag"

results = find_optimal_number_of_clusters(max_n_clusters=9, path=os.path.join(results_path, "cvi"), training_data=X_clust_scaled, seed=SEED, cov_type=covariance_type)

In [ ]:
n_components = 3

X_clust = X[selected_proteins]

scaler = StandardScaler()
X_clust_scaled = scaler.fit_transform(X_clust)

gm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=SEED).fit(X_clust_scaled)

df_baseline["cluster_n"] = gm.predict(X_clust_scaled)
df_baseline["cluster_n"]

# Relabel
df_baseline["cluster_n_bin"] = df_baseline["cluster_n"].map({0:1, 1:0, 2:1}) # 0 lowest risk, 2 highest risk
cluster_labels = {0:2, 1:1, 2:3}
df_baseline["cl_n_sorted"] = df_baseline["cluster_n"].replace(cluster_labels)

df_baseline["cluster_n_c1"] = df_baseline["cl_n_sorted"].map({1:0, 2:1, 3:0})
df_baseline["cluster_n_c2"] = df_baseline["cl_n_sorted"].map({1:0, 2:0, 3:1})

## ClusterX

In [ ]:
df_spider = df_baseline[selected_proteins +["cl_n_sorted"]]
df_spider = df_spider.reset_index(drop=True)

scaler_spider = StandardScaler()
scaler_spider.fit(df_spider[selected_proteins])
X_spider = scaler_spider.transform(df_spider[selected_proteins])
X_spider = pd.DataFrame(X_spider, columns=selected_proteins)

df_spider = pd.concat([X_spider, df_spider[["cl_n_sorted"]]], axis=1)

In [ ]:
selected_proteins_person = ['ACE2',
                            'BNP',
                            'KIM-1',
                            'REN',
                            'CD40-L',
                            'Dkk-1',
                            'Gal-9',
                            'RAGE',
                            'TRAIL-R2',
                            'CTSL1',
                            'PAPPA',
                            'ANGPT1',
                            'PGF',
                            'MMP-12',
                            'AGRP',
 ] # only to reorder

for clus in [1, 2, 3]:
    df_clus = df_spider[df_spider["cl_n_sorted"]==clus]
    metric_dict = {}
    metric_dict['group']=['TBM']
    
    for prot in selected_proteins_person:
        metric_dict[prot] = [float(df_clus[prot].mean())]
        
    df = pd.DataFrame(metric_dict)
    print(df)
    
    # ------- PART 1: Create background

    # number of variable
    categories = list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)
    #fig, ax = plt.subplots(figsize=(5, 5), layout="constrained", polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    #print(categories)
    # Draw one axe per variable + add labels
    plt.tick_params(axis='both', which='major', pad=10, direction='out', grid_linestyle='dotted', labelsize=24)
    plt.xticks(angles[:-1], categories)
    
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([-2, -1, 0, 1, 2,], 
    ["-2", "-1", "0", "1", "2"]
    , color="black", size=12)
    plt.ylim(-1.5, 1.5)
    
    #plt.gca().set_yticklabels_size('15')
    ax.tick_params(axis='both', which='major', labelsize=11)

    #highlights the reference    
    index_reference = 2
    plt.gca().get_yticklabels()[index_reference].set_size('14')
    plt.gca().get_yticklabels()[index_reference].set_weight('bold')
    ax.yaxis.get_gridlines()[index_reference].set_linestyle('-')
    ax.yaxis.get_gridlines()[index_reference].set_linewidth(1.9)
    ax.yaxis.get_gridlines()[index_reference].set_color('black')
    ax.yaxis.get_gridlines()[index_reference].set_alpha(.5)

    color_cat1 = 'royalblue' #BP and Kidney
    plt.gca().get_xticklabels()[0].set_color(color_cat1)
    plt.gca().get_xticklabels()[1].set_color(color_cat1)
    plt.gca().get_xticklabels()[2].set_color(color_cat1)
    plt.gca().get_xticklabels()[3].set_color(color_cat1)
    
    color_cat2 = 'black' # inflam and immune
    plt.gca().get_xticklabels()[4].set_color(color_cat2)
    plt.gca().get_xticklabels()[5].set_color(color_cat2)
    plt.gca().get_xticklabels()[6].set_color(color_cat2)
    plt.gca().get_xticklabels()[7].set_color(color_cat2)

    color_cat3 = 'darkcyan' #apoptosis
    plt.gca().get_xticklabels()[8].set_color(color_cat3)
    
    color_cat4 = 'grey' #autophagy
    plt.gca().get_xticklabels()[9].set_color(color_cat4)
    
    color_cat5 = 'purple' #proteolysis
    plt.gca().get_xticklabels()[10].set_color(color_cat5)
    
    color_cat6 = 'saddlebrown' #angiogenisis
    plt.gca().get_xticklabels()[11].set_color(color_cat6)
    plt.gca().get_xticklabels()[12].set_color(color_cat6)
    
    color_cat7 = 'hotpink' #celllular matrix
    plt.gca().get_xticklabels()[13].set_color(color_cat7)
    
    color_cat8 = 'goldenrod' #metabolism
    plt.gca().get_xticklabels()[14].set_color(color_cat8)

    if clus==1:
        values = df.loc[0].drop('group').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, color='darkgreen',
        linestyle='dashed', label="Cluster 1")
        #ax.fill(angles, values, color='#005C84', alpha=0.1)

    elif clus==2:
        values = df.loc[0].drop('group').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, color='darkorange',
        linestyle='dotted', label="Cluster 2")
        #ax.fill(angles, values, color='purple', alpha=0.1)

    if clus==3:
        values = df.loc[0].drop('group').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, color='darkred',
        linestyle='dashdot', label="Cluster 3")
        #ax.fill(angles, values, color='#005C84', alpha=0.1)

    plt.legend(loc='lower left', bbox_to_anchor=(1, 1), fontsize=11)
        
plt.show()

In [ ]:
shap_clusters = True

if shap_clusters:
    explainer2 = shap.Explainer(gm.predict_proba, X_clust_scaled, feature_names=intersect_features, seed=SEED)
    shaps = explainer2(X_clust_scaled)

    labels = gm.predict(X_clust_scaled)

    for i in range(len(np.unique(labels))):
        shap.summary_plot(shap_values=shaps[:, :, i], features=X_clust_scaled, 
                        max_display=18,
                        feature_names=intersect_features, show=False)
        ax = plt.gca()
        ax.set_xlim(-0.8, 0.8)
        ax.set_title(f"Cluster {cluster_labels[i]}")
        plt.show()

# C1. Phenogroups clinical validation

## Clinal characteristics comparison

In [ ]:
int_dict = {
    "age": ["mean+-std", 1, "Age, y"],
    "bmi": ["mean+-std", 2, "Body mass index, kg/m\u00b2"],
    "sbp": ["mean+-std", 2,  "Systolic pressure, mm Hg"], 
    "dbp": ["mean+-std", 2,  "Diastolic pressure, mm Hg"], 
    "pr": ["mean+-std", 2,  "Heart rate, beats/min"], 
    "smk_1": ["n_bin", 1, "Current smoking, n(%)"],
}  #examples

In [ ]:
df_aux = df_baseline.copy()
df_aux = avg_impute_characteristics(df_aux, int_dict)

df_1 = df_aux[df_aux["cl_n_sorted"]==1]
df_2 = df_aux[df_aux["cl_n_sorted"]==2]
df_3 = df_aux[df_aux["cl_n_sorted"]==3]

descriptive = show_organized_descriptive(
    [df_aux
     ], int_dict, sex_col_name, female_code, male_code, female_code)

display(descriptive)

descriptive = show_organized_descriptive(
    [df_1, df_2, df_3
     ], int_dict, sex_col_name, female_code, male_code, female_code)
descriptive.columns = ['Variable', 'Cluster 1', 'Cluster 2', 'Cluster 3']


print(mannwhitneyu(df_1['prevent_risk_ascvd'], df_2['prevent_risk_ascvd']))
print(mannwhitneyu(df_1['prevent_risk_ascvd'], df_3['prevent_risk_ascvd']))
print(mannwhitneyu(df_2['prevent_risk_ascvd'], df_3['prevent_risk_ascvd']))

print(mannwhitneyu(df_1['ins'], df_2['ins']))
print(mannwhitneyu(df_1['ins'], df_3['ins']))
print(mannwhitneyu(df_2['ins'], df_3['ins']))

#p_from_tstats(descriptive, columns={0: ['2', '5', '*'], 1: ['4', '5', '^']}, recycle_column=False)
descriptive_p = p_from_tstats(descriptive, columns={0: ['Cluster 1', 'Cluster 2', '*'], 
                                    1: ['Cluster 1', 'Cluster 3', '*'],
                                    3: ['Cluster 2', 'Cluster 3', '†']}, 
                              recycle_column=True)

## Classical survival analysis

### KM

In [ ]:
colouring = {1: "darkgreen", 2: "darkorange", 3: "darkred"}

kaplan_meier_plot(dataset=df_baseline, column="cl_n_sorted", te_column=time_out_var, e_column=event_out_var,
                      cluster_colours=colouring, 
                      path=save_results_km, 
                      max_time=18.1)

outcome_stats(df_baseline, "cl_n_sorted", t=time_out_var, e=event_out_var, spath=save_results_km)

### Cox

In [ ]:
input_vars_base = ["age", sex_col_name, "bmi", "sbp", "tchol", "trt_ht", 'hcv2', "hdm", "smk"] 

dependent_variables = input_vars_base.copy() 

df_baseline_out = df_baseline[dependent_variables + ["cl_n_sorted"] + [time_event_var, event_out_var]].dropna()

model = ProportionalHazardRatio(time_to_event_column=time_out_var, covariates=dependent_variables,
                                event_column=event_out_var, 
                                group_column="cl_n_sorted", risk_against="cluster").fit(df_baseline_out)
results = model.summary()
print(results)

# C2. Phenogroups echocardiographic characterization

## Baseline

### Echo baseline characteristics

In [ ]:
int_dict = {
    "lvidd": ["mean+-std", 2, "LV internal diameter, cm"], 
    "lvpwd": ["mean+-std", 2, "LV posterior wall diameter, cm"], 
    "rwt": ["mean+-std", 2, "Relative wall thickness"], 
    "lvmi": ["mean+-std", 2, "LV mass index, g/m^2"], 
}  # examples

In [ ]:
df_aux = df_baseline.dropna(subset=eprime_avg)
df_aux = avg_impute_characteristics(df_aux, int_dict)

print(len(df_baseline), len(df_aux))

df_1 = df_aux[df_aux["cl_n_sorted"]==1]
df_2 = df_aux[df_aux["cl_n_sorted"]==2]
df_3 = df_aux[df_aux["cl_n_sorted"]==3]


descriptive = show_organized_descriptive(
    [df_1, df_2, df_3
     ], int_dict, sex_col_name, female_code, male_code, female_code)
descriptive.columns = ['Variable', 'Cluster 1', 'Cluster 2', 'Cluster 3',  
                       ]
descriptive
descriptive_p = p_from_tstats(descriptive, columns={0: ['Cluster 1', 'Cluster 2', '*'], 
                                    1: ['Cluster 1', 'Cluster 3', '*'],
                                    3: ['Cluster 2', 'Cluster 3', '†'],

}, recycle_column=True)

### LS means

In [ ]:
df_aux = df_baseline.copy() 

In [ ]:
%%R -i df_aux -o df_lsmeans -o df_lsmeans_detailed -o df_lsmeans_pairs

library(emmeans) 
library(readxl)

set.seed(0)

dat_read <- df_aux
dat_read$cl_n_sorted <- as.factor(dat_read$cl_n_sorted)
dat_read$sex_female0 <- as.factor(dat_read$sex_female0)
dat_read$smk_1 <- as.factor(dat_read$smk_1)
dat_read$trt_ht <- as.factor(dat_read$trt_ht)

echo_cont = c() # echo variables to compare

#create data frame
df_lsmeans <- data.frame(matrix(ncol = 4, nrow = 0))

df_lsmeans_detailed <- data.frame(matrix(ncol = 4, nrow = 0))
df_lsmeans_pairs <- data.frame(matrix(ncol = 4, nrow = 0))

#provide column names
colnames(df_lsmeans) <- c('echo_var', 'cluster1', 'cluster2', 'cluster3')

for (i in 1:length(echo_cont)){
  set.seed(0)
  #print(i)
  echo_var = echo_cont[i]
  print(paste('-------------------------------------------------------------' ))
  print(paste('         -------------------', echo_var, '-------------------        ' ))
  print(paste('-------------------------------------------------------------' ))
  
  
  fit1 = lm(get(echo_var) ~ age + sex_female0 + bmi + pr + sbp + trt_ht + cl_n_sorted, data = dat_read)
  
  print(summary(fit1))
  
  res_em = emmeans(fit1, "cl_n_sorted"
                   )

  test_em <- test(res_em)
  
  comp_em <- pairs(res_em,  
                   adjust="none" 
  )
  
  res_em <- data.frame(res_em)
  print(res_em)
  test_em_df <- data.frame(test_em)
  print(test_em_df)
  comp_em_df <- data.frame(comp_em)
  comp_em_df['echo_var'] <- echo_var
  print(comp_em_df)
  
  df_lsmeans[i, 'echo_var'] <- echo_var
    
  for (ls_i in c(1, 2, 3)){
    df_lsmeans[i, paste0('cluster', res_em[ls_i, "cl_n_sorted"])] <- paste(round(res_em[ls_i, "emmean"], 3), "±", round(res_em[ls_i, "SE"]*sqrt(nrow(dat_read)),3))
  }
  
  for (ls_i in c(1, 2, 3)){
    test_em_df[ls_i, "Lower"] <-  res_em[ls_i, "lower.CL"]
    test_em_df[ls_i, "Upper"] <- res_em[ls_i, "upper.CL"]
    test_em_df[ls_i, "echo_var"] <- echo_var
  }
  
  for (ls_i in c(1, 2, 3)){
    comp_em_df[ls_i, "group1"] <- unlist(strsplit(unlist(strsplit(comp_em_df[ls_i,'contrast'], " - "))[1], "cl_n_sorted"))[2]
    comp_em_df[ls_i, "group2"] <- unlist(strsplit(unlist(strsplit(comp_em_df[ls_i,'contrast'], " - "))[2], "cl_n_sorted"))[2]
  }
  
  df_lsmeans_detailed <- rbind(df_lsmeans_detailed, test_em_df)
  df_lsmeans_pairs <- rbind(df_lsmeans_pairs, comp_em_df)

}

In [ ]:
sig_echo = set(list(df_lsmeans_pairs[df_lsmeans_pairs["p.value"]<0.05]['echo_var']))
df_lsmeans_detailed = df_lsmeans_detailed[df_lsmeans_detailed['echo_var'].isin(list(sig_echo))]

i=0

fig, geeeks = plt.subplots() 
#echo_var = 'lvidd'

for echo_var in sig_echo:
    print(echo_var)

    plt.xticks([1, 2, 3], ['Cluster 1', 'Cluster 2', 'Cluster 3'], fontsize=14)
    df_group = df_lsmeans_detailed[df_lsmeans_detailed['echo_var']==echo_var]
    plot_confidence_interval(1, [df_group.iloc[0, :]['emmean'], df_group.iloc[0, :]['Lower'], df_group.iloc[0, :]['Upper']])
    plot_confidence_interval(2, [df_group.iloc[1, :]['emmean'], df_group.iloc[1, :]['Lower'], df_group.iloc[1, :]['Upper']])
    plot_confidence_interval(3, [df_group.iloc[2, :]['emmean'], df_group.iloc[2, :]['Lower'], df_group.iloc[2, :]['Upper']])
    h = 0.01
    ypos = df_group['Upper'].max()
    ymin = df_group['Lower'].min()
    df_comp = df_lsmeans_pairs[df_lsmeans_pairs['echo_var']==echo_var]
    #print(df_comp)
    for i in range(3):
        if df_comp.iloc[i, :]['p.value'] < 0.05:
            x1 = int(df_comp.iloc[i, :]['group1'])
            x2 = int(df_comp.iloc[i, :]['group2'])
            h = 0.12*(ypos-ymin) if (x1==1 and x2==3)|(x1==3 and x2==1) else 0.05*(ypos-ymin)
            p_value = "P<0.0001" if df_comp.iloc[i, :]['p.value'] < 0.0001 else f"P={round(df_comp.iloc[i, :]['p.value'], 4):.4f}"
            #print(p_value)
            plt.text(((x1+x2)/2), ypos+h, p_value, ha='center', va='bottom', fontsize=12)
            plt.plot([x1, x1, x2, x2], 
                    [ypos+.2*h, ypos+h, ypos+h, ypos+1.5*h], lw=0.5, color='white')#just to adjust y axis
            plt.plot([x1, x1, x2, x2], 
                    [ypos+.2*h, ypos+h, ypos+h, ypos+.2*h], lw=0.5, color='black')

    plt.ylabel(int_dict[echo_var][2], fontsize=15)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.gca().yaxis.set_major_locator(ticker.LinearLocator(5))
    
    if (ypos - ymin) < 0.5:    
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_two_digits))
    elif (ypos - ymin) < 1.5:    
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_one_digit))
    else:
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_zero_digit))
    plt.show()

### LR

In [ ]:
x_cols = [] #variables to adjust

y_var_cat = "" #LV remodeling variable

df_aux_lvh = df_aux.copy()
df_aux_lvh.dropna(subset=x_cols+[y_var_cat], inplace=True)
lr_results, _, _ = backward_lr_regression(df_aux_lvh, y_var_cat, x_cols, cte=True, back=True, n_round=4)


In [ ]:
y_var_cat = "" #LVDD variable

df_aux_lvh = df_aux.copy()
df_aux_lvh.dropna(subset=x_cols+[y_var_cat], inplace=True)
lr_results, _, _ = backward_lr_regression(df_aux_lvh, y_var_cat, x_cols, cte=True, back=True, n_round=4)


# Longitudinal changes

## Delta and FU computations

In [ ]:
df_fu = pd.read_excel("") #change to FU file

## Clinical and echocardiographic characteristics

In [ ]:
int_dict_v2 = {
    "age": ["mean+-std", 1, "Age, y"],
    "bmi": ["mean+-std", 2, "Body mass index, kg/m\u00b2"],
    "sbp": ["mean+-std", 2,  "Systolic pressure, mm Hg"], 
    "dbp": ["mean+-std", 2,  "Diastolic pressure, mm Hg"], 
    "pp": ["mean+-std", 2,  "Pulse pressure, mm Hg"], 
} # examples

In [ ]:
df_1 = df_aux[df_aux["cl_n_sorted"]==1]
df_2 = df_aux[df_aux["cl_n_sorted"]==2]
df_3 = df_aux[df_aux["cl_n_sorted"]==3]


int_dict = int_dict_v2 
df_fu_aux = avg_impute_characteristics(df_fu_aux, int_dict)

df_1_fu = df_fu_aux.loc[df_aux[df_aux["cl_n_sorted"]==1].index, :]
df_2_fu = df_fu_aux.loc[df_aux[df_aux["cl_n_sorted"]==2].index, :]
df_3_fu = df_fu_aux.loc[df_aux[df_aux["cl_n_sorted"]==3].index, :]

In [ ]:
descriptive = show_organized_descriptive(
    [df_1, df_1_fu,
     ], int_dict, sex_col_name, female_code, male_code, female_code)
descriptive.columns = ['Variable', 'Baseline', 'Follow-up',  
                       ]
descriptive_p = p_from_tstats(descriptive, columns={0: ['Baseline', 'Follow-up', '*'],   
}, recycle_column=True)
descriptive_p_1 = descriptive_p.copy()


descriptive = show_organized_descriptive(
    [df_2, df_2_fu,
     ], int_dict, sex_col_name, female_code, male_code, female_code)
descriptive.columns = ['Variable', 'Baseline', 'Follow-up',  
                       ]
descriptive_p = p_from_tstats(descriptive, columns={0: ['Baseline', 'Follow-up', '*'],      
}, recycle_column=True)
descriptive_p_2 = descriptive_p.copy()


descriptive = show_organized_descriptive(
    [df_3, df_3_fu,
     ], int_dict, sex_col_name, female_code, male_code, female_code)
descriptive.columns = ['Variable', 'Baseline', 'Follow-up',  
                       ]
descriptive
descriptive_p = p_from_tstats(descriptive, columns={0: ['Baseline', 'Follow-up', '*'],     
}, recycle_column=True)
descriptive_p_3 = descriptive_p.copy()

pd.concat([descriptive_p_1, descriptive_p_2[["Baseline", "Follow-up"]], descriptive_p_3[["Baseline", "Follow-up"]]], axis=1)

### LS means

In [ ]:
%%R -i df_aux -o df_lsmeans -o df_lsmeans_detailed -o df_lsmeans_pairs

library(emmeans) 
library(readxl)

dat_read <- df_aux
dat_read$cl_n_sorted <- as.factor(dat_read$cl_n_sorted)
dat_read$lvh_up <- as.factor(dat_read$lvh_up)
dat_read$sex_female0 <- as.factor(dat_read$sex_female0)
dat_read$smk_1 <- as.factor(dat_read$smk_1)
dat_read$trt_ht <- as.factor(dat_read$trt_ht)

echo_cont = c() # delta echo variables

#create data frame
df_lsmeans <- data.frame(matrix(ncol = 4, nrow = 0))

df_lsmeans_detailed <- data.frame(matrix(ncol = 4, nrow = 0))
df_lsmeans_pairs <- data.frame(matrix(ncol = 4, nrow = 0))

#provide column names
colnames(df_lsmeans) <- c('echo_var', 'cluster1', 'cluster2', 'cluster3')

for (i in 1:length(echo_cont)){
  set.seed(0)
  #print(i)
  echo_var = echo_cont[i]
  print(paste('-------------------------------------------------------------' ))
  print(paste('         -------------------', echo_var, '-------------------        ' ))
  print(paste('-------------------------------------------------------------' ))
  echo_var_aux = unlist(strsplit(echo_var, "d_"))[2]
  
  #examples
  if (echo_var=="d_lvidd"){
    fit1 = lm(d_lvidd ~ lvidd + age +  sex_female0 + bmi + d_bmi + pr + d_pr + sbp + d_sbp + started_or_continued_trt_htn + cl_n_sorted, data = dat_read)
  } else if (echo_var=="d_ivrt"){
    fit1 = lm(d_ivrt ~ ivrt + age + sex_female0 + bmi + d_bmi + pr + d_pr + sbp + d_sbp + started_or_continued_trt_htn + cl_n_sorted, data = dat_read)
  }
  
  print(summary(fit1))
  
  res_em = emmeans(fit1, "cl_n_sorted"#, weights='proportional'
                   )

  test_em <- test(res_em)
  
  comp_em <- pairs(res_em, adjust="none" 
  )
  

  res_em <- data.frame(res_em)
  test_em_df <- data.frame(test_em)
  comp_em_df <- data.frame(comp_em)
  comp_em_df['echo_var'] <- echo_var
  
  
  df_lsmeans[i, 'echo_var'] <- echo_var
  
  
  for (ls_i in c(1, 2, 3)){
    df_lsmeans[i, paste0('cluster', res_em[ls_i, "cl_n_sorted"])] <- paste(round(res_em[ls_i, "emmean"], 3), "±", round(res_em[ls_i, "SE"]*sqrt(nrow(dat_read)),3))
  }
  
  for (ls_i in c(1, 2, 3)){
    test_em_df[ls_i, "Lower"] <-  res_em[ls_i, "lower.CL"]
    test_em_df[ls_i, "Upper"] <- res_em[ls_i, "upper.CL"]
    test_em_df[ls_i, "echo_var"] <- echo_var
  }
  
  for (ls_i in c(1, 2, 3)){
    comp_em_df[ls_i, "group1"] <- unlist(strsplit(unlist(strsplit(comp_em_df[ls_i,'contrast'], " - "))[1], "cl_n_sorted"))[2]
    comp_em_df[ls_i, "group2"] <- unlist(strsplit(unlist(strsplit(comp_em_df[ls_i,'contrast'], " - "))[2], "cl_n_sorted"))[2]
  }
  
  df_lsmeans_detailed <- rbind(df_lsmeans_detailed, test_em_df)
  df_lsmeans_pairs <- rbind(df_lsmeans_pairs, comp_em_df)

}

In [ ]:
sig_echo = set(list(df_lsmeans_pairs[df_lsmeans_pairs["p.value"]<0.05]['echo_var']))
df_lsmeans_detailed = df_lsmeans_detailed[df_lsmeans_detailed['echo_var'].isin(list(sig_echo))]

i=0

fig, geeeks = plt.subplots() 

for echo_var in sig_echo:
    print(echo_var)

    plt.xticks([1, 2, 3], ['Cluster 1', 'Cluster 2', 'Cluster 3'], fontsize=14)
    #plt.title('Confidence Interval')
    df_group = df_lsmeans_detailed[df_lsmeans_detailed['echo_var']==echo_var]
    plot_confidence_interval(1, [df_group.iloc[0, :]['emmean'], df_group.iloc[0, :]['Lower'], df_group.iloc[0, :]['Upper']])
    plot_confidence_interval(2, [df_group.iloc[1, :]['emmean'], df_group.iloc[1, :]['Lower'], df_group.iloc[1, :]['Upper']])
    plot_confidence_interval(3, [df_group.iloc[2, :]['emmean'], df_group.iloc[2, :]['Lower'], df_group.iloc[2, :]['Upper']])
    h = 0.01
    ypos = df_group['Upper'].max()
    ymin = df_group['Lower'].min()
    df_comp = df_lsmeans_pairs[df_lsmeans_pairs['echo_var']==echo_var]
    #print(df_comp)
    for i in range(3):
        if df_comp.iloc[i, :]['p.value'] < 0.05:
            x1 = int(df_comp.iloc[i, :]['group1'])
            x2 = int(df_comp.iloc[i, :]['group2'])
            h = 0.12*(ypos-ymin) if (x1==1 and x2==3)|(x1==3 and x2==1) else 0.05*(ypos-ymin)
            p_value = "P<0.0001" if df_comp.iloc[i, :]['p.value'] < 0.0001 else f"P={round(df_comp.iloc[i, :]['p.value'], 4):.4f}"
            plt.text(((x1+x2)/2), ypos+h, p_value, ha='center', va='bottom', fontsize=12)
            plt.plot([x1, x1, x2, x2], 
                    [ypos+.2*h, ypos+h, ypos+h, ypos+1.5*h], lw=0.5, color='white')#just to adjust y axis
            plt.plot([x1, x1, x2, x2], 
                    [ypos+.3*h, ypos+h, ypos+h, ypos+.3*h], lw=0.5, color='black')

    plt.ylabel(delta_dict[echo_var][2], fontsize=15)

    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.gca().yaxis.set_major_locator(ticker.LinearLocator(5))
    
    if (ypos - ymin) < 0.5:    
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_two_digits))
    elif (ypos - ymin) < 1.5:    
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_one_digit))
    else:
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(fmt_zero_digit))
    plt.show()

### LR

In [ ]:
x_cols = [] #input variables

y_var_cat = "d_lv_remol_inc"

df_aux_lvh_up = df_aux[(df_aux["d_lv_remol_inc"]==1)|(df_aux['d_lv_remol_same_down']==1)]
lr_results, _, _ = backward_lr_regression(df_aux_lvh_up, y_var_cat, x_cols, cte=True, back=True, n_round=4)


In [ ]:
y_var_cat = "d_lvdd_inc"

df_aux_lvdd = df_aux[(df_aux["d_lvdd_inc"]==1)|(df_aux['d_lvdd_same_down']==1)]
_, _, _ = backward_lr_regression(df_aux_lvdd, y_var_cat, x_cols, cte=True, back=True, n_round=4)